### 번역자: 고려대학교 수학과(12) 최시현
[AiKorea 번역 프로젝트](https://github.com/aikorea/cs231n)를 이어받아서 진행하였습니다. 기존 AiKorea에서 이미 번역되어있는 CourseNote의 경우 유사한 부분이 많습니다. <br /> 
(깨진 삽화를 추가하거나 개인적으로 번역이 매끄럽지 못하다 생각하는 부분은 수정하였습니다.) <br />
문제가 될시에는 자진 삭제하겠습니다. 오류 수정 및 내용에 대한 기타 문의 사항은 passkmla@naver.com으로 연락주시기 바랍니다.

## 선형 분류(Linear Classification)

지난 섹션에서는 특정 카테고리의 집합으로부터 하나의 라벨을 이미지에 할당하는 문제인 이미지 분류에 대해 소개하였다. <br /> 
또한, 학습 데이터셋에 있는 (라벨링 된)이미지들과 비교하여 가까이 있는 이미지들의 라벨을 활용하는 k-Nearest Neighbor(kNN) 분류기에 대해 설명하였다. <br /> 앞서 살펴 보았듯이 kNN은 몇 가지 단점이 있다.
- 이 분류기는 모든 학습 데이터를 기억해야 하고, 나중에 테스트 데이터와 비교하기 위해 저장해 두어야 한다. <br />
이것은 메모리 공간 관점에서 매우 비효율적이고, 일반적인 데이터셋들은 용량이 기가바이트 단위를 쉽게 넘기는 것이 많기 때문에 문제가 된다.
- 테스트 이미지를 분류할 때 모든 학습 이미지와 다 비교를 해야 하기 때문에 계산량/시간이 매우 많이 소요된다.

#### Overview

이번 노트에서는 이미지 분류를 위한 보다 강력한 방법들을 발전시켜 나갈 것이고, 이는 나중에 뉴럴 네트워크와 컨볼루션 뉴럴 네트워크로 확장될 것이다. <br />
이 방법들은 두 가지 중요한 요소가 있다. <br /> 
데이터를 클래스 스코어로 매핑시키는 스코어 함수, 그리고 예측한 스코어와 실제 라벨과의 차이를 정량화 해주는 손실함수가 그 두 가지이다. <br />
우리는 이를 최적화 문제로 바꾸어서 스코어 함수의 파라미터들에 대한 손실 함수를 최소화 할 것이다.

## 이미지에서 라벨 스코어로의 파라미터화된 매핑

먼저, 이미지의 픽셀 값들을 각 클래스에 대한 신뢰도 점수(Confidence score)로 매핑시켜주는 스코어 함수를 정의한다. <br />
여기서는 구체적인 예시를 통해 각 과정을 살펴볼 것이다. 이전 노트에서처럼, 학습 데이터셋 이미지들인 ${ x }_{ i }\in { R }^{ D }$가 있고, <br />
각각이 해당 라벨 ${ y }_{ i }$를 갖고 있다고 하자. 여기서 $i=1...N$ 그리고 ${ y }_{ i }\in \left\{ 1...K \right\}$이다. <br />
즉, 학습할 데이터 N개가 있고 (각각은 D차원의 벡터이다) 총 K개의 서로 다른 카테고리 (클래스)가 있다. <br />
예를들어, CIFAR-10 에서는 N=50,000 개의 학습 데이터 이미지들이 있고, 각각은 D = 32x32x3 = 3072 픽셀로 이루어져 있으며, <br /> 
(dog, cat, car 등) 10개의 서로 다른 클래스가 있으므로 K=10이다. <br />
이제 이미지의 픽셀값들을 클래스 스코어로 매핑해 주는 스코어 함수 $f:{ R }^{ D }\mapsto { R }^{ K }$을 아래에 정의할 것이다.

#### 선형 분류기(Linear Classifier)

이 파트에서는 가장 단순한 함수라고 할 수 있는 선형 매핑 함수로 시작할 것이다.

<center>$f({ x }_{ i },W,b)=W{ x }_{ i }+b$</center>

위 식에서, 우리는 각 이미지 ${ x }_{ i }$의 모든 픽셀들이 [Dx1]모양을 갖는 하나의 열 벡터로 평평하게 펼쳐졌다고 가정하였다. <br />
[KxD]차원의 행렬 $W$와 [Kx1]차원의 벡터 b는 이 함수의 파라미터이다. CIFAR-10에서 ${ x }_{ i }$는 i번째 이미지의 <br />
모든 픽셀을 [3072x1]크기로 평평하게 펼쳐진 열 벡터가 될것이고, $W$는 [10x3072], b는 [10x1]이어서 3072개의 숫자가 <br />
함수의 입력(이미지 픽셀 값들)으로 들어와 10개의 숫자가 출력(클래스 스코어)으로 나오게 된다. <br />
$W$안의 파라미터들은 보통 Weight라고 불리고, $b$는 bias 벡터라 불리는데, <br /> 
그 이유는 실제 입력 데이터인 ${ x }_{ i }$와의 아무런 상호 작용 없이 출력 스코어 값에는 영향을 주기 때문이다. <br />
그러나 보통 일반적으로 사람마다 Weight와 Parameter 두 개의 용어를 혼용해서 사용하는 경우가 많다. 

여기서 몇 가지 짚고 넘어갈 점이 있다.
- 먼저, 한 번의 행렬곱 $W{ x }_{ i }$ 만으로 10개로의 다른 분류기(각 클래스마다 하나씩)를 병렬로 계산하는 효과를 나타내고 있다 <br />
이 때 $W$행렬의 각 열이 각각 하나의 분류기가 된다.
- 또한, 여기서 입력 데이터(${ x }_{ i }$, ${ y }_{ i }$)는 주어진 값이고 고정되어 있지만, 파라미터들인 $W$, $b$의 세팅은 우리가 조절할 수 있다. <br />
우리의 최종 목표는 전체 학습 데이터에 대해서 우리가 계산할 스코어 값들이 실제(ground truth) 라벨과 가장 잘 일치하도록 이 파라미더의 값들을 정하는 것이다. <br />
이후(아래)에 자세한 방법에 대해 다룰 것이지만, 직관적으로 간략하게 말하자면 올바르게 잘 맞춘 클래스가 틀린 클래스들보다 더 높은 스코어를 갖도록 조절한 것이다.
- 이러한 방식의 장점은, 학습 데이터가 파라미터들인 $W$, $b$를 학습하는데 사용되지만 학습이 끝난 이후에는 학습된 파라미터들만 남기고, <br /> 
학습에 사용된 데이터셋은 더이상 필요가 없다는 (따라서 메모리에서 지워버려도 된다는) 점이다. <br /> 
그 이유는 새로운 테스트 이미지가 입력으로 들어올 때 위의 함수에 의해 스코어를 계산하고, 계산된 스코어를 통해 바로 분류되기 때문이다.
- 마지막으로, 테스트 이미지를 분류할 때 행렬곱 한 번과 덧셈 한번을 하는 계산만 필요하다는 점이다. <br /> 
이것은 테스트 이미지를 모든 학습 이미지와 비교하는 것에 비하면 매우 빠르다.

> 컨볼루션 뉴럴 네트워크는 위에서 살펴본것 처럼 이미지 픽셀들을 스코어로 매핑시킬 것이다. <br />
그러나 매핑 함수($f$)는 더 복잡하고 많은 파라미터를 포함하고 있다.

## 선형 분류기 분석하기

선형 분류기는 클래스 스코어를 이미지의 모든 픽셀 값들의 가중치 합으로 스코어를 계산하고, 이 때 각 픽셀의 3개의 색 채널을 모두 고려한다. <br />
이 때 각 가중치(파라미터, Weight)에 어떤 값을 주느냐에 따라 스코어 함수는 이미지의 특정 위치에서 특정 색깔을 선호하거나 선호하지 않거나 (가중치 값의 부호에 따라) 할 수있다. <br />
예를 들어, "ship" 클래스는 이미지의 가장자리 부분에 파란색이 많은 경우에 (강, 바다 등의 물에 해당하는 색) 스코어 값이 더 높아질 것이라고 추측해 볼 수 있을 것이다. <br />
즉, "ship"분류기는 파란색 채널의 파라미터(Weights)들이 양의 값을 갖고 (파란색이 존재하는 것이 ship의 스코어를 증가시키도록), 빨강/초록색 채널에는 음의 값을 <br />
갖는 파라미터들이 많을 것이라고 (빨간색/초록색의 존재는 ship의 스코어를 감소시키도록)예상 할 수 있다.
___

<img src="./Images/1.png" width=700 />

이미지에서 클래스 스코어로의 매핑 예시. <br />
시각화를 위해서, 이미지가 픽셀 4개 만으로 이루어져 있고 (색 채널도 고려하지 않고, 단일 채널이라고 생각하자). <br />
3개의 클래스가 있다고 하자. (빨강(cat), 초록(dog), 파랑(ship)클래스). (주. 여기서의 색깔은 3개의 클래스를 나타내기 위함이고 RGB채널과 전혀 관련이 없다.) <br />
이제 이미지 픽셀들을 펼쳐서 열 벡터로 만들고 각 클래스에 대해 행렬곱을 수행하면 스코어 값을 얻을 수 있다. <br />
여기서 정해준 파라미터 $W$값들은 매우 안 좋은 예시이다. <br />
현재의 파라미터로는 고양이(cat)이미지를 매우 낮은 cat 스코어를 갖도록 한다. <br />
이 경우, 현재의 파라미터 값은 우리가 개(dog)이미지를 보고 있다고 생각하고 있다.
___

#### 이미지를 고차원 공간 상의 점으로 매핑하는 것에 대한 분석

이미지들을 고차원 열 벡터로 펼쳤기 때문에, 우리는 각 이미지를 이 고차원 공간 상의 하나의 점으로 생각할 수 있다. <br />
(e.g: CIFAR-10 데이터 셋의 각 이미지는 32x32x3 개의 픽셀로 이루어진 3072-차원 공간 상의 한 점이 된다.) <br />
마찬가지로 생각하면, 전체 데이터셋은 라벨링된 고차원 공간 상의 점들의 집합이 될 것이다. <br />
위에서 각 클래스에 대한 스코어를 이미지의 모든 픽셀에 대한 가중치 합으로 정의했기 때문에, <br /> 
각 클래스 스코어는 이 공간상에서의 선형 함수값이 된다. 3072-차원 공간을 시각화 할 수 없지만, <br /> 
2차원으로 축소시켰다고 상상해보면 우리의 분류기가 어떤 행동을 하는지를 시각화하려고 시도해볼 수 있을 것이다. 
___

<img src="./Images/2.png" width=400 />

이미지 공간의 시각화. 각 이미지는 하나의 점에 해당되고, 3개의 분류기가 표시되어 있다. <br />
자동차(car)분류기(빨간색)를 예로 들어보면, 빨간색 선이 이 공간 상에서 car 클래스에 대해 스코어 값이 0이 되는 모든 점을 나타낸 것이다. <br /> 
빨간색 화살표는 스코어가 증가하는 방향을 나타낸 것으로, 빨간색 선의 오른쪽에 있는 점들은 <br />
양의(그리고 선형적으로 증가하는) 스코어 값을 가질 것이고, 왼쪽의 점들은 음의(그리고 선형적으로 감소하는) 스코어를 가질 것이다. 
___

위에서 살펴보았듯이, $W$의 각 행은 각각의 클래스를 구별하는 분류기이다. 각 행에 있는 숫자들을 기하학적으로 해석해보자면, <br />
우리가 $W$의 하나의 행을 바꾸면 픽셀 공간에서 해당하는 선이 다른 방향으로 회전할 것이다. 반면에, bias인 $b$는 분류기가 그 선들을 평행이동 할 수 있도록 해준다. <br />
특히, bias가 없다면 ${ x }_{ i }$=0 가 입력으로 들어왔을때 파라미터의 값들에 상관없이 항상 스코어가 0이 될 것이고, 모든 (분류) 선들이 원점을 지나야만 할 것이다.

#### 템플릿 매칭으로서 선형 분류기에 대한 해석

파라미터 $W$에 대해 다른 방식으로 해석해보면, $W$의 각 행은 각 클래스별 템플릿(또는 프로토타입)에 해당한다. <br />
이미지의 각 클래스 스코어는 각 템플릿과 이미지의 내적을 통해 계산되고, 이 스코어를 기준으로 가장 잘 맞는 것이 무엇인지 정해진다. <br />
즉, 선형 분류기가 결국 템플릿 매칭을 하고 있고, 각 템플릿이 학습을 통해 배워진다고 할 수 있다. <br />
또다른 방식으로 생각해보면, 우리는 Nearest Neighbor와 비슷한 것을 하고 있는데,<br /> 
수 천 장의 학습 이미지를 갖고 있지 않고 각 클래스마다 한 장의 이미지만 사용한다고 볼 수 있다. <br />
(다만, 그 이미지가 필연적으로 학습 데이터셋에 실제로 존재하는 이미지일 필요는 없다.) <br /> 
이 때 거리로는 L1이나 L2거리를 사용하지 않고 내적을 사용한다.
___

<img src="./Images/3.png" width=800 />

약간의 선행학습: CIFAR-10 데이터셋에 학습된 파라미터들의 시각화 예시. <br />
예를 들어 ship 템플릿을 보면 예상할 수 있듯이, 많은 수의 파란색 픽셀들로 이루어져 있다. <br />
이 템플릿은 배가 바다 위에 떠있는 이미지와 내적을 통해 비교되었을 때, 높은 스코어 값을 가질 것이다.
___

추가적으로, horse 템플릿은 머리가 두 개인 말이 있는 것처럼 보이는데, 이것은 데이터셋 안에 왼쪽을 보고 있는 말과 오른쪽을 보고 있는 말이 섞여있기 떄문이다. <br />
선형 분류기는 말에 대한 이 두 가지 형태를 하나의 템플릿으로 합친 것을 확인할 수 있다. <br />
이와 비슷한 현상으로, car 분류기는 모든 방향 및 색깔의 자동차 모양들을 하나의 템플릿으로 합쳐 놓았다. 특히, 이 템플릿이 결과적으로 붉은 색을 띄는 것으로 보아 <br />
CIFAR-10 데이터 셋에는 다른 색깔에 비해 빨간색 자동차가 더 많다는 점을 알 수 있다. 선형 분류기는 여러 가지 색깔의 자동차를 제대로 분류하기에는 모델이 <br />
너무 단순하지만, 나중에 배울 뉴럴 네트워크는 이를 해결할 수 있다. 조금만 미리 살펴보자면, 뉴럴 네트워크는 히든 레이어의 각 뉴런들이 특정 자동차 타입 <br />
(e.g: 왼쪽을 바라보고 있는 초록색 자동차, 정면을 보고 있는 파란색 차, 등등)을 검출하도록 할 수 있고, 다음 레이어의 뉴런들이 이 정보들을 종합하여 각각의 <br />
자동차 타입 검출기의 점수의 가중치 합을 통해 보다 정확한 (자동차에 대한) 스코어를 계산할 수 있다.

#### Bias 트릭

다음 내용으로 넘어가기 전에, 두 파라미터 $W$, $b$를 하나로 표현하는 간단한 트릭을 소개한다. 앞에서 스코어 함수는 아래와 같이 정의되었다.

<center>$f({ x }_{ i },W,b)=W{ x }_{ i }+b$</center>

앞으로 내용을 전개해 나갈 때 두 가지 파라미터를 (bias $b$와 weight $W$)매번 동시에 고려해야 한다면 표현이 번거로워진다. <br />
트릭은 이 두 파라미터들을 하나의 행렬로 합치고, ${ x }_{ i }$에 대하여 항상 1의 값을 갖는 한 차원 -a default bias dimension- 을 늘리는 방식이다. <br />
이 한 차원 추가하는 것으로, 새 스코어 함수는 행렬곱 한 번으로 단순화된다.

<center>$f({ x }_{ i },W)=W{ x }_{ i }$</center>

CIFAR-10 예제에서, ${ x }_{ i }$는 이제 [3072x1]대신에 [3073x1]이 되고 (새로 추가된 extra-dimension은 상수 1이 됨), <br /> 
$W$는 [10x3072]대신에 [10x3073]이 된다. $W$의 extra column은 bias $b$에 대응된다. <br />
명확한 이해를 위해 아래의 그림을 살펴보자.
___

<img src="./Images/4.png" width=600 />

Bias trick의 시각화. 행렬곱을 하고 Bias 벡터를 더하는 것과 Input 벡터에 상수 1을 더하고 Weight 행렬에 Bias 열을 더하는 것은 동등하다. <br />
그러므로 만약 모든 입력 벡터에 1을 더한다면, Weight와 Bias 두 개의 행렬을 이용하는 대신에 하나의 Weight 행렬로 학습할 수 있다. 
___

#### 이미지 자료 전처리

위의 예제에서는 범위가 0~255인 픽셀 값을 사용했다. 기계학습에서는 input features(이미지의 경우 모든 pixel들이 feature가 됨)의 <br />
정규화(normalization)를 수행하는 것이 일반적인 관례이다. 특히 모든 features로부터 평균을 빼서 자료를 가운데로 이동시키는 것이 중요하다. <br />
이미지의 경우, pixel의 범위를 [-127,127]로 근사한 이미지들을 얻기 위해 해당 학습 이미지들의 모든 pixel로부터 평균을 계산하고, <br /> 
해당 이미지의 각 pixel에서 이것을 빼준다. 더 일반적인 전처리는 각 입력 특징 값의 범위를 [-1,1]로 하는 것이다. <br />
그중에서 특히 평균을 0으로 Centering하는 것은 매우 중요하다. <br /> 
하지만 이것의 당위성을 이해하기 위해서는 Decent Gredient의 역할을 제대로 이해하여야 하므로 나중에 더 살펴보자.

## 손실 함수(Loss function)

이전 장에서는 가중치들의 집합 $W$에 의한 parameterized를 통해 pixel값으로부터 클래스 스코어로의 함수를 정의했다. <br />
또한, 주어진 $({ x }_{ i }, { y }_{ i })$를 조정하지 않고, 가중치만을 조정하였다. <br />
그리고 예측된 클래스 스코어가 학습 데이터의 실제 라벨과 일치하는지에 대해 가중치를 세팅하고자 한다. <br />

예를 들어, 고양이 이미지의 예제로 돌아가보면, 이 이미지로부터 "고양이", "개", "배" 각각의 클래스에 대해 점수가 매겨졌고,<br /> 
이 예제에서는 특정 가중치들의 집합이 전혀 좋지 않음을 보았다. <br /> 
Pixel들로부터 얻은 고양이 점수는 -96.8로 다른 클래스(개: 437.9, 배; 61.85)에 비해 매우 낮았다. <br />
우리는 손실함수(비용 함수 또는 Objective)처럼 이러한 결과물의 잘못된 정도를 측정할 것이다. <br /> 
직관적으로보면, 만약 학습 자료를 잘못 분류하는 이라면 손실 함수의 값은 높을 것이고 잘 분류하는 것이라면 손실 함수의 값은 낮을 것이다.

#### 다중 클래스 서포트 벡터 머신 손실(Multiclass Support Vector Machine Loss)

손실 함수의 세부 사항을 정의하는 몇가지 방법이 있다. <br />  
첫 번째 예제로서, Multiclass Support Vector Machine Loss이라고 불리는 손실을 개발할 것이다. <br />
SVM 손실은 SVM이 올바른 클래스의 각각의 이미지에 대하여 올바르지 않은 클래스들과 비교하여 어떤 고정된 마진 $\Delta$보다 더 높은 점수를 갖기를 바라는 것으로 설정되어 있다. <br />

좀 더 정확하게 살펴보면, i번째 예제에 대해 이미지의 픽셀들 ${x}_{i}$와 올바른 클래스의 index를 나타내는 라벨 ${y}_{i}$가 주어졌다. <br /> 
이 스코어 함수는 픽셀들을 가지고 클래스 스코어 벡터 $f({x}_{i}, W)$를 계산한다. 이제 이 스코어 함수를 s로 줄여서 쓰자. <br />
예를 들어, j번 째 클래스에 대한 점수는 j번 째 성분 ${s}_{j}=f({x}_{i}, W)$이다. <br />
i번째 예제에 대한 Multiclass SVM Loss는 다음과 같이 나타낼 수 있다.

<img src="./Images/5.png" width=400 />

#### 예제

이것이 어떻게 작동하는지 보기위해 예제를 살펴보자. <br />
세개의 클래스가 다음과 같은 점수로 주어졌다고 가정하자. <br /> 
$s=[13, -7, 11]$ 그리고 첫번째 클래스가 실제 클래스이다. (${y}_{i}=0$) <br />
$\Delta$(곧 공부하게될 하이퍼 파라미터)는 10으로 가정하자. <br />
위의 표현식을 올바르지 않은 모든 클래스 ($j\neq {y}_{i}$)에 대해 더하면 다음의 두 항을 얻을수 있다. 

<img src="./Images/6.png" width=600 />

[-7-13+10]이 음수이기 때문에 0을 threshold로 하는 $max(0,-)$함수에 의해 첫 항은 0이 된다. 우리는 이들 쌍에 대해 0 손실을 얻었다. <br />
그 이유는, 올바른 클래스의 스코어(13)가 올바르지 않은 클래스의 스코어(-7)와 비교하여 적어도 마진 10보다 크기 때문이다. <br /> 
실제로 이 차이는 20이고 이는 마진 10보다 크다. <br />
이 예제에서 SVM은 적어도 마진 10보다 작은 것들만 고려한다. 마진보다 차이가 큰 것들은 max 연산에 의해 0이 된다. <br />
두 번째 항은 [11-13+10]을 계산하면 8이 된다. 올바른 클래스가 올바르지 않은 클래스보다 더 높은 점수(13>11)이지만 그 차이가 마진 10보다 크지 않다. <br />즉, 차이가 2이므로 손실은 8이 나온다. (i.e: 마진을 만족시키려면 얼마만큼 더 높아야 하는가) <br /> 
요약하자면, SVM 손실 함수는 올바른 클래스 ${y}_{i}$의 점수가 올바르지 않은 클래스 점수보다 적어도 $\Delta$(델타)보다는 크기를 바란다. <br /> 
이 경우가 아니라면 우리는 손실을 누적할 것이다.

선형 스코어 함수($f({x}_{i},W)=W{x}_{i}$)로 손실 함수를 다시 쓰면 다음과 같다.

<img src="./Images/7.png" width=400 />

${w}_{j}$는 $W$의 j번째 행이고, 열로 전치된다. 그러나 더 복잡한 형태의 스코어 함수가 되는 것을 고려하면 이것이 반드시 필요한 것은 아니다. 

이 장을 마치기 전 짚고 넘어가야할 함수가 있다. 그것은 바로 0에서 threshold를 가지는 $max(0,-)$이다. 이는 hinge loss라고도 불린다. <br />
사람들은 종종 hinge loss SVM(또는 L2-SVM)대신에 ${max(0,-)}^{2}$을 사용한다. 이는 선형적이기보다 이차원적이고 violated 마진에 대해 더 큰 페널티를 부여한다. <br />
보통 제곱하지 않은 버전이 더 일반적이지만, 어떤 데이터 셋에 대해서는 squared hinge loss가 더 잘 작동한다.
이는 교차검증을 통해 결정될 수 있다.

> 손실 함수는 트레이닝 데이터셋에 대한 우리의 Unhappiness를 측정한다.


___
<img src="./Images/8.png" width=800 />

Multiclass Support Vector Machine은 올바른 클래스의 점수가 다른 클래스의 점수보다 적어도 델타 만큼의 마진보다는 크길 원한다. <br />
만약 어느 클래스가 빨간 지역에 있다면, 이들은 손실로 누적된다. 그렇지 않으면 손실은 0이 된다. <br />
우리의 목적은 학습 데이터에서 모든 예제에 대해 이 제약을 만족시키는 가중치를 찾는 것이고, 가능한 총 손실은 낮길 바란다.
___

#### 정규화(Regularization)

위에서 언급한 손실 함수는 한 가지 버그를 가지고 있다. <br />
어떤 데이터 셋을 가지고 있고, 모든 예제를 올바르게 분류하는 파라미터의 집합 $W$가 있다고 가정하자. <br /> 
(모든 스코어, 모든 마진이 맞아 떨어진다. 즉, 모든 $i$에 대해 ${L}_{i}=0$) <br />
이 문제는 집합 $W$가 필연적으로 고유하지 않다는 것이다. 즉, 예제를 올바르게 분류하는 많은 유사한 $W$가 있을 수 있다. <br />
이를 바라보는 한가지 쉬운 방법은, 만약 어떤 파라미터의 집합$W$가 모든 예제들을 올바르게 분류한다면, $\lambda W$ 또한($\lambda$>1라면) 손실을 0으로 만들 것이다. <br />
왜냐하면 이 변화는 모든 스코어 magnitude를 균등하게 늘리고 따라서 그들의 절대적 차이 또한 균등하게 늘리기 때문이다. <br />
예를 들어, 만약 올바른 클래스와 가자우가까운 잘못된 클래스 사이의 점수 차이가 15라면 W의 모든 요소에 2를 곱해줌으로써 차이는 30이 된다.

따라서 우리는 이와 같은 모호성을 제거하고 특정 가중치 집합 $W$에 대해 몇가지 성향이 표현되길 원한다. <br />
우리는 정규화 패널티(Regularization penalty) $R(W)$와 함께 손실 함수를 확장함으로써 이를 해낼 수 있다. <br />
가장 일반적인 정규화 패널티(Regularization penalty)는 모든 파라미터에 대해 elementwise 이차항 페널티를 통해 큰 가중치를 억제하는 L2-norm이다.

<img src="./Images/9.png" width=250 />

위의 표현에서 우리는 $W$의 모든 제곱 element를 다 더한다. 이 정규화(regularizaion) 함수는 데이터의 함수는 아니다. 이것은 단지 가중치에만 기반한다. <br />
데이터 손실(모든 예제에 대한 ${L}_{i}$의 평균 손실)과 정규화(regularizaion) 손실의 두 요소로 만들어진 정규화(regularizaion) 페널티를 적용한 <br /> 
Multiclass Support Vector Machine Loss을 다음과 같이 표현 할 수 있다.

<img src="./Images/10.png" width=300 />

이 형태를 확장하여 쓰면 다음과 같다.

<img src="./Images/11.png" width=800 />

N은 학습 예제의 수이다. 위 식에서 볼 수 있듯이, loss objective에 하이퍼파라미터 $\lambda$에 의해 가중화된 정규화(regularizaion) 페널티를 추가한다. <br />
이 하이퍼 파라미터를 설정하는 간단한 방법은 없다. 일반적으로 교차검증에 의해 결정된다.

위에서 언급하였듯이, 이 방법은 정규화(regularizaion) 페널티를 포함하여 많은 바람직한 특징들이 존재한다. 이것들에 대해서는 이후 장에서 살펴볼 것이다. <br />
예를들어 L2 페널티를 포함하는 것은 SVM에서 max margin Property로 이어진다는 것이 증명되었다. <br />
(자세한 내용에 대해 더 공부하고 싶다면 [CS229](http://cs229.stanford.edu/notes/cs229-notes3.pdf)의 lecture notes를 참고하여라)

가장 흥미로운 특성은 큰 가중치에 페널티를 주는 것이 일반화(Generalization)를 향상시키는 효과가 있다는 것이다. <br />
왜냐하면, 어떤 입력 차원도 그 자체로 모든 스코어에 큰 영향을 줄 수는 없기 때문이다. <br />
예를 들어, 입력 벡터 $x$=[1, 1, 1, 1]에 대하여, ${w}_{1}$=[1, 0, 0, 0], ${w}_{2}$=[0.25, 0.,25, 0.25, 0.25]가 있다고 하자. <br />
그러면 ${w}_{1}x = {w}_{2}x = 1$, 즉 두 가중치 벡터에 대한 내적은 같은 값을 가진다. 그러나 ${w}_{1}$의 L2 패널티는 1이고, ${w}_{2}$의 L2패널티는 단지 0.25이다. <br />
그러므로 ${w}_{2}$의 가중치 벡터가 더 작고 더 분산되어있다. L2 페널티는 더 작고 더 분산된 가중치 벡터를 선호하기 때문에, <br />
최종 분류기는 적은 수의 입력 차원들이 매우 큰 것 보다는 모든 입력 차원들의 값이 작고 분산된 값인 것을 선호한다. <br />
나중에 더 자세히 살펴보겠지만, 이런 효과는 테스트 이미지에 대한 분류기의 일반화(generalization) 성능을 향상시키고, 과적합(overfit(을 줄이는 효과가 있다.

bias들은 가중치들과는 같은 효과를 갖지 않는다. 왜냐하면, 가중치와는 다르게 입력 차원의 영향 강도를 조절할 수 없기 때문이다. <br />
그러므로, 보통 bias $b$가 아닌 가중치 $W$만 정규화(regularize)하는 것이 일반적이다. 그러나 실제로 이것은 무시할만한 정도의 효과만을 가진다. <br /> 
정규화(regularization) 페널티 때문에 정확히 0.0의 손실을 얻는 것은 불가능 하다. 이것은 가중치가 정확히 $W$=0일때만 가능하다. 

#### 코드

다음은 정규화(regularization)과정이 없는 손실 함수가 파이썬으로 구현한 것이다. Unvectorized 와 half-vectorized 형식으로 되어있다.

In [ ]:
def L_i(x, y, W):
  """
  unvectorized version. Compute the multiclass svm loss for a single example (x,y)
  - x is a column vector representing an image (e.g. 3073 x 1 in CIFAR-10)
    with an appended bias dimension in the 3073-rd position (i.e. bias trick)
  - y is an integer giving index of correct class (e.g. between 0 and 9 in CIFAR-10)
  - W is the weight matrix (e.g. 10 x 3073 in CIFAR-10)
  """
  delta = 1.0 # see notes about delta later in this section
  scores = W.dot(x) # scores becomes of size 10 x 1, the scores for each class
  correct_class_score = scores[y]
  D = W.shape[0] # number of classes, e.g. 10
  loss_i = 0.0
  for j in xrange(D): # iterate over all wrong classes
    if j == y:
      # skip for the true class to only loop over incorrect classes
      continue
    # accumulate loss for the i-th example
    loss_i += max(0, scores[j] - correct_class_score + delta)
  return loss_i

def L_i_vectorized(x, y, W):
  """
  A faster half-vectorized implementation. half-vectorized
  refers to the fact that for a single example the implementation contains
  no for loops, but there is still one loop over the examples (outside this function)
  """
  delta = 1.0
  scores = W.dot(x)
  # compute the margins for all classes in one vector operation
  margins = np.maximum(0, scores - scores[y] + delta)
  # on y-th position scores[y] - scores[y] canceled and gave delta. We want
  # to ignore the y-th position and only consider margin on max wrong class
  margins[y] = 0
  loss_i = np.sum(margins)
  return loss_i

def L(X, y, W):
  """
  fully-vectorized implementation :
  - X holds all the training examples as columns (e.g. 3073 x 50,000 in CIFAR-10)
  - y is array of integers specifying correct class (e.g. 50,000-D array)
  - W are weights (e.g. 10 x 3073)
  """
  # evaluate loss over all examples in X without using any for loops
  # left as exercise to reader in the assignment

이 장에서는 SVM 손실이 학습 데이터에 대한 예측과 실제 라벨이 얼마나 일치하는지에 대해 측정하는 한 방법임을 배운다. <br />
또한, 학습 자료에 대한 좋은 예측을 하는 것은 손실을 최소화하는 것이다.

> 이제 우리가 해야할 것은 손실을 최소화하는 가중치를 찾는 방법을 알아내는 것이다.

## 실제 고려사항(Practical Considerations)

#### 델타 설정하기

이제 우리는 하이퍼 파라미터 $\Delta$와 이것의 설정에 대해서 다룬다. 어떤 값으로 설정되어야하고, 이 값에 대하여 교차 검증을 하여햐 할까? <br />
이 하이퍼 파라미터는 모든 케이스에서 $\Delta$=0.1로 안전하게 설정할 수 있다고 증명되었다. <br /> 하이퍼 파라미터 $\Delta$와$\lambda$는 서로 다른 두 하이퍼 파라미터처럼 보인다. <br /> 
그러나 실제로 이 둘은 같은 Trade off를 가진다. 이 Trade off는 목적 함수(objective)에서 데이터 손실과 정규화(Regularization) 손실에 대한 것이다. <br />
이것을 이해하기 위한 열쇠는 가중치 $W$의 magnitude가 스코어들에 직접적인 영향을 끼친다는 것이다. <br />
$W$안에 모든 값을 축소해서 넣으면 이에 따라 모든 스코어의 차이는 줄어들 것이고, 가중치들의 magnitude를 키운다면 그 차이는 늘어날 것이다. <br />
그러므로, 스코어들 사이의 특정한 값은(e.g: $\Delta$=1 이나 $\Delta$=100) 어떤 측면에서는 전혀 의미가 없다. 왜냐하면 가중치는 임의로 줄이거나 늘리거나 할 수 있기 때문이다. <br />
따라서, 정규화 강도 $\lambda$를 통해 얼마나 많이 가중치가 증가하는 것을 허용할 것인지가 진짜 trade off이다.

#### 이항 서포트 벡터 머신(Binary Support Vector Machine)과의 관계

이전에 살펴본 예제에 대하여, i번째 예제에 대한 손실을 다음과 같이 쓸 수 있다.

<img src="./Images/12.png" width=300 />

여기서 C는 하이퍼 파라미터이고, ${y}_{i}\in ${-1, 1}이다. <br />
오직 두 클래스만을 갖는 특정한 케이스에 대하여 이항 SVM을 특별한 케이스로서 포함하는 위 공식(SVM loss)을 이 섹션에서 사용할 수 있다. <br />
만약 우리가 두 개의 클래스만 가지고 있다면, 위에서 표현된 이항 SVM loss를 이용하여 손실을 줄일 수 있다. <br />
또한 이항 SVM loss공식에서의 C와 위 공식(SVM loss)에서의 $\lambda$는 같은 trade off를 조절하고 상호 관계는 $C\propto \frac { 1 }{ \lambda }$로 나타낼 수 있다. 

#### 이 외: Optimization in primal

만약 SVM에 대한 사전 지식과 함께 이 수업에 왔다면, 아마도 kernels, duals, SMO 알고리즘 등등... 에 대해 들어보았을 수도 있다. <br />
이 수업에서는 항상 unconstrained primal form에서 목적 함수(objective)를 최적화 한다. 이들 중 많은 목적 함수(objective)들이 미분 가능하지 않다. <br />
(e.g: max(x,y)함수는 미분 불가능 하다. 왜냐하면 x=y일 때 구부러지기 때문이다) <br />
그러나 실제에서는 subgradient를 사용하기 때문에 문제가 되지 않는다.

#### 이 외: 다른 Multiclass SVM formulations

이 장에서의 다중 클래스 SVM 표현(Multiclass SVM formulations)은 Multi class에 대한 SVM을 공식화하는 몇 가지 방법 중 하나에 불과하다. <br />
다른 예로서 일반적으로 사용되는 form은 다른 모든 클래스에 대하여 독립적인 이항 SVM을 학습하는 One Vs All(OVA) SVM 이다. <br />
이와 관련되어는있지만, 실제로 덜 일반적인 모델로는 All Vs All(AVA)전략도 있다. <br />
위에서 배운 formulation은 OVA(이 버전은 OVA로는 할 수 없는, 데이터 손실을 0으로 만들 수 있는 multi class 데이터 세트를 만들 수 있다는 점에서, 관심있는 경우 논문을 읽어보아라)보다 더 강력한 버전인 [ Weston and Watkins 1999 (pdf)](https://www.elen.ucl.ac.be/Proceedings/esann/esannpdf/es1999-461.pdf)을 따른다. <br />
마지막으로는, Structured SVM은 올바른 클래스의 스코어와 올바르지 않은 클래스들 중 가장 높은 스코어(runner up)의 마진을 최대화한다. <br />
이 공식들간의 차이를 이해하는 것은 이 수업의 범위를 넘는다. 위에 제시된 version은 실제로 사용하기에 꽤 안전한 방법이지만, 가장 단순한 OVA 전략 또한 잘 작동 할 것이다. <br />
(as also argued by Rikin et al. 2004 in [In Defense of One-Vs-All Classification (pdf)](http://www.jmlr.org/papers/volume5/rifkin04a/rifkin04a.pdf))

#### 소프트맥스 분류기

SVM은 보편적으로 사용되는 두 개의 분류기 중에 하나이다. <br />
다른 잘 알려진 선택은 소프트맥스 분류기이다. 이전에 이항 로지스틱 회귀 분석에 대해 들어본적이 있다면 , <br /> 
소프트 맥스 분류기는 다중 클래스에 대해 이를 일반화(generalize)한 형태로 확장한 것이라고 볼 수 있다. <br />
Output으로 각 클래스에 대한 점수 $f(x_i,W)$를 다루는 SVM과 달리, <br /> 
소프트맥스 분류기는 좀 더 직관적인 output (정규화(normailzied)된 클래스의 확률들)을 주고, 이는 확률적 해석을 갖는다. <br /> 
소프트맥스 분류기에서도 이 함수를 $f(x_i,W)=Wx_i$로 매핑하는 것은 다르지 않지만, 이들 점수를 각 클래스에 대한 정규화되지(normailized) 않은 로그 확률처럼 해석하고, <br /> 
hinge loss를 크로스 엔트로피 손실(Cross-entropy loss)로 교체한다. 따라서 다음과 같은 형태를 갖는다.

<img src="./Images/13.png" width=500 />

위에서 사용한 $f_j$표시는 클래스 스코어 $f$벡터의 j번째 요소를 의미한다. <br />
이전처럼, 데이터셋에 대한 전체 손실은 정규화(Regularization)항 $R(W)$와 모든 테스트 예제에 대한 $L_i$의 평균이다. <br />
이 함수 $f_{ j }(z)=\frac { { e }^{ { z }_{ j } } }{ \sum _{ k }^{  }{ { e }^{ { z }_{ k } } }  } $를 소프트맥스 함수라 부른다. <br />
임의의 실제 값 스코어($z$에서)들의 벡터를 취하여 이들을 0과 1사이의 벡터로 압축시켜서 이 벡터들의 총 합이 1이 되도록 만든다. <br />
소프트맥스 함수에 수반되는 크로스 엔트로피 손실은 처음본다면 어렵게 보일 수도 있지만, 사실은 그리 어렵지 않다.

#### 정보 이론 관점(Information thoery view)

실제 분포 $p$와 측정된 분포 $q$사이의 크로스 엔트로피는 다음과 같이 정의 된다.

<img src="./Images/14.png" width=250 />

소프트맥스 분류기는 따라서 실제 분포와 측정된 클래스 확률들($q=\frac { { e }^{ { f }_{ { y }_{ i } } } }{ \sum _{ j }^{  }{ { e }^{ { f }_{ j } } }  } $)사이의 크로스 엔트로피를 최소화한다. <br />
이 해석에서 분포는 모든 확률 질량이 올바른 클래스에 있다고 가정한다.(i.e p=[0, ..., 1, ..., 0]는 $y_i$번째만 1을 포함한다) <br />
또한, 크로스 엔트로피는 엔트로피의 항과 Kullback-leibler 발산 $H(p,q)=H(p)+{ D }_{ KL }(p\parallel q)$과 델타 함수 p의 엔트로피는 0으로 쓰여질 수 있기 때문에 <br /> 
이것은 두 분포 사이(a measure of distance)의 KL발산을 최소화하는 것과 동등하다. <br /> 
즉, 크로스 엔트로피 목적 함수(objective)는 모든 질량에 대해 올바른 답을 갖는 예측 분포를 원한다.

#### 확률적 해석

<img src="./Images/15.png" width=200 />

위의 식을 주어진 이미지 $x_i$와 parameterized된 $W$에 대해, 올바른 라벨 $y_i$에 할당된 정규화(normalized)된 확률처럼 해석할 수 있다. <br />
소프트맥스 분류기는 출력 벡터 $f$ 안의 스코어를 정규되지 않은(unnormalized) 로그 확률로 해석한다. <br />
이 값들을 지수화하여 확률을 부여하고, 확률의 합이 1이 되도록 정규화(normalize)하기 위해 나누기를 수행한다. <br />
확률적 해석 관점에서 보면, 올바른 클래스의 로그 Likelihood를 최소화하는 것으로 최대 우도 추정치(Maximum Likelihood Estimation(MLE))를 수행하는 것처럼 해석할 수 있다. <br /> 
이 관점의 좋은 점은, 전체 손실(full loss)에서 정규화(regularized)항 $R(W)$를 가중치 행렬 $W$에 대해 가우시안 사전 분포에서 온 것처럼 해석할 수 있고, <br /> 
MLE대신 Maximum a posterior(MAP)추정을 수행할 수 있다. 위의 해석들이 직관적 해석을 도와 줄 수 있지만, 세부사항은 이 수업의 범위를 넘는다.

#### Pratical issues: 수치적 안정성(Numeric stability)

실제로 소프트맥스 함수를 계산하기 위한 코드를 작성할 때 ${ e }^{ { f }_{ { y }_{ i } } }$ 그리고 $\sum _{ j }^{  }{ { e }^{ { f }_{ j } } } $는 지수화가 되기 때문에 매우 커진다. <br />
큰 수를 나누는 것은 수치적으로 안정적이지 않을 수 있다. 그래서 정규화(normalization) 트릭을 사용하는 것이 중요하다. <br />
만약 우리가 상수가 $C$를 분자와 분모에 곱해주면, 다음과 같은 표현을 얻을 수 있다. <br />

<img src="./Images/16.png" width=300 />

여기서 $C$값은 자유롭게 선택할 수 있다.. <br />
이 방법을 사용하여도 결과 자체에는 어떠한 변화도 가져오지 않기 때문에 계산의 수치적 안정성을 향상시키기 위해서 이 값을 사용할 수 있다. <br />
$C$에 대한 일반적인 선택은 $\log { C } =-\max _{ j }{ { f }_{ j } } $이다. <br />
이를 위해서는 벡터 $f$안에 값들을 이동시키기만 하면 된다. 그래서 가장 큰 값이 0이 된다. 코드로 보자

In [ ]:
f = np.array([123, 456, 789]) # example with 3 classes and each having large scores
p = np.exp(f) / np.sum(np.exp(f)) # Bad: Numeric problem, potential blowup

# instead: first shift the values of f so that the highest number is 0:
f -= np.max(f) # f becomes [-666, -333, 0]
p = np.exp(f) / np.sum(np.exp(f)) # safe to do, gives the correct answer

#### Possibly confusing naming conventions

SVM 분류기는 hinge Loss나 때때로 max-margin Loss를 사용한다. 소프트맥스 분류기는 크로스 엔트로피 손실을 사용한다. <br />
소프트맥스 분류기는 크로스 엔트로피 손실이 적용 가능할 수 있도록 각 클래스의 합이 1이 되도록 정규화(normalize)하는 소프트맥스 함수로부터 그 이름을 얻었다. <br />

## SVM vs Softmax

아래의 그림이 소프트맥스와 SVM 분류기 사이의 구별을 명확히 하는데 도움을 줄 것이다.
___

<img src="./Images/17.png" width=600 />

하나의 자료값에 대한 SVM과 소프트맥스 분류기 사이의 차이에 대한 예제 <br />
두 케이스에서 우리는 같은 스코어 벡터 $f$를 계산한다. 이 차이는 $f$에서 스코어의 해석에이다. <br />
SVM은 이들을 클래스 스코어로 해석하고, 손실 함수가 올바른 클래스(클래스2, 파랑)이 다른 클래스보다 마진 이상 더 큰 점수를 얻기를 권장한다. <br />
소프트맥스 분류기는 SVM과 다르게 이 점수들을 각 클래스에 대한 비정규화(unnormalized)된 로그 확률로 해석하고, 올바른 클래스의 정규화(normalized) 로그 확률이 높기를 권장한다. <br />
이 예제에 대한 최종 손실은 SVM에서는 1.58이고 소프트맥스 분류기에서는 1.04이지만 이들 숫자는 비교할 수 없다. <br />
이들은 단지 같은 분류기와 같은 자료내에서의 관계만 의미가 있기 때문이다.
___

#### 소프트맥스 분류기는 각 클래스에 대한 "확률"을 제공한다

보정되지 않아 모든 클래스에 대한 점수를 해석하기 쉽지 않은 SVM과 달리, 소프트맥스 분류기는 각 라벨에 대한 확률을 계산해준다. <br />
예를 들어, 이미지가 주어졌을때 SVM분류기는 우리에게 "고양이", "개", "배" 클라스에 대해 각각 [12.5, 0.6, -23.0]의 점수를 준다. <br />
소프트맥스 분류기는 이 세 라벨의 확률을 계산할 수 있다. [0.9, 0.09, 0.01] <br />
그러나 이 확률이라는게 얼마나 뾰족한지 얼마나 퍼져있을지는 정규화(regularization) 강도 $\lambda$에 달려있다. <br />
예를 들어, 정규화되지 않은(unnormalized) 로그 확률이 세 클래스에 대해 [1, -2, 0]이라고 가정하자. <br />
소프트맥스 함수는 다음과 같이 계산한다.

<img src="./Images/18.png" width=500 />

지수화한 다음 합이 1이 되도록 정규화(normalize)한다. 만약 정규화(regularize) 강도 $\lambda$가 크다면, 가중치 $W$는 더 큰 페널티를 받을 것이고, <br />
이것은 가중치를 더 작게 만들것이다. 예를 들어, 가중치를 반으로 줄여보자 ([0.5, -1, 0]) 그러면 소프트맥스 함수는 다음과 같이 계산된다. <br />

<img src="./Images/19.png" width=500 />

확률은 이제 더 퍼져있게 된다. <br />
게다가, 매우 강한 정규화(regularization) $\lambda$에 의해 가중치들이 매우 작은 숫자로 향하는 극한으로가는 상황에서는 이 출력 확률들은 거의 동등해 진다. <br /> 
이런 이유로 소프트맥스 분류기에 의해 계산된 확률들을 순서로 해석하는 경우에는 SVM처럼 신뢰할만 하지만 절대적인 숫자들은 그렇지 않다. 

#### 실제에서, SVM과 소프트맥스는 대게 비슷하다

SVM과 소프트맥스 사이의 성능 차이는 매우 작다. 사용자의 목적에 따라서 어떤 분류기가 더 잘 작동할 수 있다. <br />
SVM은 더 지역 지향적(local objective)인데 이것은 버그 혹은 특징으로서 비춰질 수 있다. <br />
예제에서 점수가 [10, -2, 3] 얻어졌고 첫 번째 클래스가 옳다고 하자. <br />
SVM(바람직한 마진 $\Delta$=1)은 올바른 클래스는 이미 다른 클래스에 비해 마진보다 더 높은 스코어를 얻었고 이것은 손실 0으로 계산될 것이다. <br />
SVM은 개별 값의 세부사항은 고려하지 않는다. <br />
그러나 이 시나리오는 소프트맥스 분류기와는 동등하지 않다. 소프트맥스 분류기에서는 [10, 9, 9]는 [10, -100, -100]보다 더 높은 손실이 누적될 것이다. <br />
즉, 소프트맥스 분류기는 이것이 만들어내는 점수에 대해 절대 행복하지 않다. <br />
올바른 클래스는 항상 높은 확률을 가지고, 올바르지 않은 클래스는 항상 낮은 확률을 가지며 손실은 항상 향상된다. <br />
그러나 SVM은 일단 마진만 만족하면 행복하고 이들 제약 뒤에 실제 점수에 대해서는 세세하게 관여하지 않는다. <br />
예를 들어, 자동차 분류기는 트럭들로부터 차를 분류하는 어려운 문제에 대해 대부분의 노력을 사용하지 개구리의 예제에 영향을 받지 않는다. <br />
즉 데이터 구름(data cloud)이 완전히 다른 측면으로 클러스터 되어있을 것이므로 이는 이미 낮은 점수가 개구리 예제에 대하여 할당되어 있다는 뜻이다. 
___

#### 상호작용 가능한 웹 데모(Interactive web demo)

<img src="./Images/20.png" width=600 />

선형분류기에 대한 직관적인 이해를 돕기위해 상호작용 가능한 웹 데모를 작성하였다. <br />
이 데모는 2차원 자료에 대해 3가지 분류기를 사용하여 이번 장에서 논의한 손실 함수를 시각화 한다. <br />
이 데모는 또한 다음 장에서 더 자세히 논의할 최적화에 대해 보여준다.

## Summary

In summary,

- We defined a score function from image pixels to class scores <br /> 
(in this section, a linear function that depends on weights $W$ and biases $b$).
- Unlike kNN classifier, the advantage of this parametric approach is that once we learn the parameters we can discard the training data. <br /> 
Additionally, the prediction for a new test image is fast since it requires a single matrix multiplication with $W$, <br />not an exhaustive comparison to every single training example.
- We introduced the bias trick, which allows us to fold the bias vector into the weight matrix <br /> 
for convenience of only having to keep track of one parameter matrix.
- We defined a loss function (we introduced two commonly used losses for linear classifiers: the SVM and the Softmax) <br /> 
that measures how compatible a given set of parameters is with respect to the ground truth labels in the training dataset. <br /> 
We also saw that the loss function was defined in such way that making good predictions on the training data is equivalent to having a small loss. <br />

We now saw one way to take a dataset of images and map each one to class scores based on a set of parameters, <br /> 
and we saw two examples of loss functions that we can use to measure the quality of the predictions. <br /> 
But how do we efficiently determine the parameters that give the best (lowest) loss? <br /> 
This process is optimization, and it is the topic of the next section.

#### Further Reading

These readings are optional and contain pointers of interest. <br />
- [Deep Learning using Linear Support Vector Machines](https://arxiv.org/abs/1306.0239) from Charlie Tang 2013 presents some results claiming that the L2SVM outperforms Softmax.